# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# 역 정보 수집

In [ ]:
def SubwayInfoCrawling():
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    driver = webdriver.Chrome("chromedriver.exe", options=options)
    driver.get('http://www.seoulmetro.co.kr/kr/cyberStation.do')

    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    mapinfo = soup.find('div', 'mapInfo')
    lines = mapinfo.find_all('li')

    output = pd.DataFrame()
    for i in range(len(lines)):
        st_line = lines[i].span.text
        st_list = lines[i].div.text.split(',')
        for j in range(len(st_list)):
            st_name = st_list[j].strip()
            unit = pd.DataFrame({'st_name':[st_name],
                                 'st_line':[st_line]})
            output = pd.concat([output,unit], axis=0)

    output = output.reset_index(drop=True)
    driver.close()
    return output

In [ ]:
st_info = SubwayInfoCrawling()

## 데이터 전처리
### 호선명

In [ ]:
print(st_info['st_line'].unique())

In [ ]:
line_dict = {
    '분당':'분당선',
    '신분당':'신분당선',
    '경의중앙':'경의중앙선',
    '용인경전철':'에버라인',
    '우이신설경전철':'우이신설선',
    '김포':'김포골드라인'    
}

In [ ]:
st_info['st_line'] = st_info['st_line'].replace(line_dict)

### 역명

In [ ]:
st_info.loc[st_info['st_name']=='4·19민주묘지','st_name'] = '4.19민주묘지'
st_info.loc[st_info['st_name']=='사우(김포시청)','st_name'] = '사우'
st_info['st_name'] = st_info['st_name'].apply(lambda x: x if x[-1]=='역' else x + '역')

# 역 주소 크롤링

In [ ]:
import getpass
user_name = getpass.getuser()
kakao_api_key = pd.read_csv(f'C:/Users/{user_name}/Google Drive/Secret/kakao.csv')
# 제 고유의 카카오 api 입니다. 헤더로 GET 해야 응답을 받을 수 있습니다.

In [ ]:
headers = {'Authorization':f"{kakao_api_key['rest_api'][0]}"}

In [ ]:
def Geocoding(st_name, st_line):
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={st_name} {st_line}'
    response = requests.get(url, headers=headers)
    lat = response.json()['documents'][0]['y']
    lng = response.json()['documents'][0]['x']
    return [lat,lng]

In [ ]:
st_info['coordinates'] = st_info.apply(lambda x: Geocoding(x['st_name'], x['st_line']), axis=1)

In [ ]:
st_info['st_lat'] = st_info['coordinates'].apply(lambda x: x[0])
st_info['st_lng'] = st_info['coordinates'].apply(lambda x: x[1])
st_info = st_info.drop(columns='coordinates')

# 역 정보 전처리

In [ ]:
st_info['st_lat'] = st_info['st_lat'].apply(lambda x: float(x))
st_info['st_lng'] = st_info['st_lng'].apply(lambda x: float(x))
print(st_info.info())

In [ ]:
print(st_info[st_info['st_name']=='신촌역'])
print(st_info[st_info['st_name']=='서울역'])

- 신촌역과 서울역(경의중앙선)은 같은 역이 아닙니다. 경의중앙선 역은 이름에 (경의중앙선)을 붙여 구분하도록 합니다.

In [ ]:
st_info.loc[(st_info['st_name']=='신촌역')&(st_info['st_line']=='경의중앙선'), 'st_name'] = '신촌역(경의중앙선)'
st_info.loc[(st_info['st_name']=='서울역')&(st_info['st_line']=='경의중앙선'), 'st_name'] = '서울역(경의중앙선)'

- 환승역을 고려하여 역 이름은 같으나 호선이 다른 경우에는 위치를 하나로 통일할 필요가 있습니다.

In [ ]:
st_location = st_info.groupby('st_name')['st_lat','st_lng'].agg(np.mean)

In [ ]:
new_st_info = st_location.merge(st_info[['st_name','st_line']], how='left', on='st_name')

- 환승역의 개수를 넣어주도록 합니다.

In [ ]:
new_st_info = new_st_info.merge(new_st_info.groupby('st_name')['st_line'].count().to_frame('st_count'),
                                how='left', on='st_name')

In [ ]:
new_st_info = new_st_info[['st_line','st_name','st_lat','st_lng','st_count']].sort_values('st_line').reset_index(drop=True)
new_st_info.head()
new_st_info.to_csv('./data/subway_location_info.csv', index=False)